In [1]:
# Libraries
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
from tqdm import tqdm
import time
import numpy as np

# Preliminaries

from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

## Dataset

In [2]:
source_folder = 'data/sentiment_treebank'
destination_folder = 'logs/sentiment_analysis'

In [3]:
os.makedirs(destination_folder, exist_ok=True)

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Model parameter
MAX_SEQ_LEN = 64
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)



label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('text', text_field)]

# TabularDataset

train, valid = TabularDataset.splits(path=source_folder, train='train.csv', validation='eval.csv',
                                           format='CSV', fields=fields, skip_header=True)


# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)

## BERT

In [6]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name, num_labels=3)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [7]:
# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']


In [8]:
# Training Function

def train_model(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 5,
          eval_every = len(train_iter),
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    running_step = 0
    valid_running_loss = 0.0
    valid_running_step = 0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        print(f'---> epoch {epoch+1} <---')
        
        
        time.sleep(0.5)
        loader = tqdm(train_loader, postfix={'Epoch': epoch+1})
        model.train()
        
        for (labels, text), _ in loader:
            
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            text = text.type(torch.LongTensor)  
            text = text.to(device)
            output = model(text, labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1
            running_step += 1
            
            loader.set_postfix({
                'Epoch': epoch+1,
                'Train loss': running_loss / running_step,
            }, refresh=True)
            
            
        # evaluation step
        time.sleep(0.5)
        loader = tqdm(valid_loader, postfix={'Epoch': epoch + 1,}, colour='green')
        
        model.eval()
        with torch.no_grad():
            label_preds = []
            # validation loop
            for (labels, text), _ in loader:
                labels = labels.type(torch.LongTensor)           
                labels = labels.to(device)
                text = text.type(torch.LongTensor)  
                text = text.to(device)
                output = model(text, labels)
                loss, label_pred = output

                valid_running_loss += loss.item()
                valid_running_step += 1

                label_preds.extend((torch.argmax(label_pred, dim=1) == labels).cpu().numpy() * 1)

                loader.set_postfix({
                    'Epoch': epoch+1,
                    'Valid loss': valid_running_loss / valid_running_step,
                    'Valid score': np.mean(label_preds)
                }, refresh=True)

        # evaluation
        average_train_loss = running_loss / eval_every
        average_valid_loss = valid_running_loss / len(valid_loader)
        train_loss_list.append(average_train_loss)
        valid_loss_list.append(average_valid_loss)
        global_steps_list.append(global_step)

        # resetting running values
        running_loss = 0.0          
        running_step = 0

        valid_running_loss = 0.0
        valid_running_step = 0



        # print progress
        time.sleep(0.5)
        
        loader.write('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
              .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                      average_train_loss, average_valid_loss))

        # checkpoint
        if best_valid_loss > average_valid_loss:
            best_valid_loss = average_valid_loss
            save_checkpoint(file_path + '/' + 'model_bert.pt', model, best_valid_loss)
            save_metrics(file_path + '/' + 'metrics_bert.pt', train_loss_list, valid_loss_list, global_steps_list)

            loader.write('*** save ***')

        time.sleep(0.5)
    
    save_metrics(file_path + '/' + 'metrics_bert.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    print('Finished Training!')

In [9]:
model = BERT().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
train_model(model=model, optimizer=optimizer)

---> epoch 1 <---


100%|██████████| 149/149 [00:09<00:00, 15.05it/s, Epoch=1, Valid loss=0.753, Valid score=0.73] 


Epoch [1/5], Step [593/2965], Train Loss: 0.7845, Valid Loss: 0.7527
Model saved to ==> logs/sentiment_analysis/bert/model.pt
Model saved to ==> logs/sentiment_analysis/bert/metrics.pt
*** save ***
---> epoch 2 <---


100%|██████████| 149/149 [00:10<00:00, 14.82it/s, Epoch=2, Valid loss=0.753, Valid score=0.73] 


Epoch [2/5], Step [1186/2965], Train Loss: 0.7827, Valid Loss: 0.7526
Model saved to ==> logs/sentiment_analysis/bert/model.pt
Model saved to ==> logs/sentiment_analysis/bert/metrics.pt
*** save ***
---> epoch 3 <---


...

Epoch [3/5], Step [1779/2965], Train Loss: 0.7808, Valid Loss: 0.7535
---> epoch 4 <---


100%|██████████| 149/149 [00:10<00:00, 14.46it/s, Epoch=4, Valid loss=0.753, Valid score=0.73] 


Epoch [4/5], Step [2372/2965], Train Loss: 0.7828, Valid Loss: 0.7531
---> epoch 5 <---


100%|██████████| 149/149 [00:10<00:00, 14.49it/s, Epoch=5, Valid loss=0.753, Valid score=0.73] 


Epoch [5/5], Step [2965/2965], Train Loss: 0.7809, Valid Loss: 0.7532
Model saved to ==> logs/sentiment_analysis/bert/metrics.pt
Finished Training!


In [11]:
model.eval()
for (labels, text), _ in valid_iter:
    labels = labels.type(torch.LongTensor)           
    labels = labels[:1] .to(device)
    text = text.type(torch.LongTensor)  
    text = text[:1].to(device)
    
    _, output = model(text, labels)
    print(output.argmax(dim=1), labels)
    

tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([1], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([1], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([1], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([0], de